In [1]:
%%html
<style> .container { width: 100% !important; }</style>

!pip install --upgrade pip

In [52]:
import pandas as pd
pd.set_option('max_colwidth', 200)  # Set a very large value
pd.set_option('max_row', 200)  # Set a very large value

def add_borders(val):
    return 'border: 1px solid black;'

style_df = lambda df : df.style.applymap(add_borders)

In [3]:
locations_df = pd.read_json("data2.json")

----- Data Example

In [56]:
locations_df.head(1).T

,0
loc_name,tredp
loc_class,redshift
loc_directory,
loc_remote_node,10.242.112.153
loc_remote_login,isc_dev
loc_remote_pwd,!{zSCdIs9t}!
loc_remote_port,8443
loc_db_name,"JSON=[[""odbc_driver"",""/opt/amazon/redshiftodbc/lib/64/libamazonredshiftodbc64.so""],[""db_node"",""us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com""],[""db_port"",""5439""],[""db_name"",..."
loc_db_user,504005930/!{zSCeRnNVccKz8Sbf}!
loc_description,tredp


In [63]:
pd.DataFrame (locations_df.columns, columns=["columns"])

,columns
0,loc_name
1,loc_class
2,loc_directory
3,loc_remote_node
4,loc_remote_login
5,loc_remote_pwd
6,loc_remote_port
7,loc_db_name
8,loc_db_user
9,loc_description


In [10]:
set (locations_df.dbdesc)

{'Finance Nprod', 'US hvr prod', 'Us innovation', 'dev', 'fin-prod'}

---------------

-- Analysis --

--------------

In [14]:
locations_prod_df = locations_df[locations_df['dbdesc'].isin(['US hvr prod','fin-prod'])]

-- how many locations are using our agents

locations_prod_df[['loc_remote_node','loc_remote_port']].drop_duplicates().sort_values(by=['loc_remote_node','loc_remote_port'])

In [82]:
print ('Number of locations per hvr meta database')

style_df(
locations_prod_df.groupby(['dbdesc', 'database'])['loc_name'].count().reset_index().set_index(['dbdesc','database'])
)


Number of locations per hvr meta database


In [84]:
print ('number of agents per node (represented as port) - count of locations')

style_df (
locations_prod_df.pivot_table(index='loc_remote_node', columns=['loc_remote_port'], values='loc_name', aggfunc='count').fillna('').astype(str).apply(lambda x: x.str.rstrip('.0'))
)



number of agents per node (represented as port) - count of locations


loc_remote_port,0,4343,4344,6080,8080,8088,8443,9090,9995,18081
loc_remote_node,,,,,,,,,,
,22,,,,,,,,,
10.229.1.113,,1,,,,,,,,
10.229.2.236,,,,,4,,,,,
10.242.109.196,,448,,8,5,,1,,1,
10.242.109.235,,,,,3,,,,,
10.242.109.242,,,,,,,4,26,4,1
10.242.109196,,3,,,,,,,,
10.242.112.130,,34,,,,,,,,
10.242.112.153,,6,,1,,86,1,186,,


In [ ]:
-- rac://cgehedq2-scan.am.health.ge.com:1525 repeated multiple times is it same agent ?

In [54]:
style_df (
            locations_prod_df.pivot_table(index='loc_remote_node', columns=['loc_class'], values='loc_name', aggfunc='count').fillna('').astype(str).apply(lambda x: x.str.rstrip('.0'))
         )

loc_class,file,greenplum,kafka,mysql,oracle,postgresql,redshift,salesforce,sqlserver,teradata
loc_remote_node,,,,,,,,,,
,95,,3,2,88,24,,3,,5
10.229.1.113,1,,,,,,,,,
10.229.2.236,2,,,,,,2,,,
10.242.109.196,341,2,4,,29,51,28,,,17
10.242.109.235,,,,,,,3,,,
10.242.109.242,12,,,,,19,4,,,
10.242.109196,3,,,,,,,,,
10.242.112.130,,,,,,1,,3,3,
10.242.112.153,159,,2,,2,48,43,,,17


In [58]:
style_df (
locations_prod_df.pivot_table(index='loc_remote_node', columns=['dbdesc','database'], values='loc_name', aggfunc='count').fillna('').astype(str).apply(lambda x: x.str.rstrip('.0'))
)